In [2]:
!pip install tensorflow==2.9.1 tensorflow-gpu==2.9.1 opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time 
import mediapipe as mp

# Mediapipe Holictics

In [3]:
mp_holistics_model = mp.solutions.holistic
mp_drawing_utilities = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img.flags.writeable = False  #img will be non writeable
    results = model.process(img) #making prediction
    img.flags.writeable = True   #img will be writeable again
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img, results

In [5]:
def draw_landmarks(img,results):
    mp_drawing_utilities.draw_landmarks(img, results.face_landmarks, mp_holistics_model.FACEMESH_TESSELATION)
    mp_drawing_utilities.draw_landmarks(img, results.pose_landmarks, mp_holistics_model.POSE_CONNECTIONS)
    mp_drawing_utilities.draw_landmarks(img, results.left_hand_landmarks, mp_holistics_model.HAND_CONNECTIONS)
    mp_drawing_utilities.draw_landmarks(img, results.right_hand_landmarks, mp_holistics_model.HAND_CONNECTIONS)

In [6]:
def draw_styled_landmarks(img,results):
    mp_drawing_utilities.draw_landmarks(img, results.face_landmarks, mp_holistics_model.FACEMESH_TESSELATION,
                                       mp_drawing_utilities.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                       mp_drawing_utilities.DrawingSpec(color=(80,255,121),thickness=1,circle_radius=1)
                                       )
    mp_drawing_utilities.draw_landmarks(img, results.pose_landmarks, mp_holistics_model.POSE_CONNECTIONS,
                                       mp_drawing_utilities.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                       mp_drawing_utilities.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)
                                       )
    mp_drawing_utilities.draw_landmarks(img, results.left_hand_landmarks, mp_holistics_model.HAND_CONNECTIONS,
                                       mp_drawing_utilities.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=4),
                                       mp_drawing_utilities.DrawingSpec(color=(121,44,250),thickness=1,circle_radius=2)
                                       )
    mp_drawing_utilities.draw_landmarks(img, results.right_hand_landmarks, mp_holistics_model.HAND_CONNECTIONS,
                                       mp_drawing_utilities.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                                       mp_drawing_utilities.DrawingSpec(color=(245,66,230),thickness=1,circle_radius=1)
                                       )

In [7]:
#mp_holistics_model.POSE_CONNECTIONS

In [8]:
# mp_drawing_utilities.draw_landmarks??

In [9]:
cap = cv2.VideoCapture(0)

#Access mediapipe model
with mp_holistics_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        success, frame = cap.read()

        img, results = mediapipe_detection(frame, holistic)
        draw_landmarks(img,results)
#         draw_styled_landmarks(img,results)
        
        frame = cv2.resize(frame, (860,640))
        cv2.imshow("Output", img)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [10]:
draw_landmarks(frame, results)
# plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

# Keypoints value

In [11]:
print(results.pose_landmarks.landmark[0].x,
results.pose_landmarks.landmark[0].y,
results.pose_landmarks.landmark[0].z,
results.pose_landmarks.landmark[0].visibility)

0.4036298394203186 0.4421319365501404 -0.4747008681297302 0.9999925494194031


In [12]:
all_landmarks_pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    all_landmarks_pose.append(test)

#we have pose and all_landmarks_pose same arrays just diff way of allocation

In [13]:
pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [18]:
print(f"Pose : {pose}\n\n\nFace : {face}\n\n\nLeft hand : {lh}\n\n\nRight hand : {rh}")

In [15]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

# Folder for collection of Images

In [16]:
DATA_PATH = os.path.join('MP_DATA')
actions = np.array(["Hello","ThankYou","ILoveYou"])

# no of frames that we collect, ie 
#30 videos woth of data
no_sequence = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [17]:
for action in actions:
    for seq in range(no_sequence):
        try:
            os.makedirs(os.path.join(DATA_PATH, action,str(seq)))
        except:
            pass
        

# Collecting Frames to be processed into actions

In [99]:
cap = cv2.VideoCapture(0)

#Access mediapipe model
with mp_holistics_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        for seq in range(no_sequence):
            for frame_num in range(sequence_length):

                success, frame = cap.read()

                img, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(img,results)
                if frame_num == 0:
                    cv2.putText(img, 'Starting Collection',(120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1,cv2.LINE_AA)
                    cv2.putText(img, f'Collecting frames for {action} Video Number {seq}',(120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1,cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(img,f'Collecting frames for {action} Video Number {seq}', (15,12), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(seq), str(frame_num))
                np.save(npy_path, keypoints)
                    
                frame = cv2.resize(frame, (860,640))
                cv2.imshow("Output", img)
                
                if cv2.waitKey(1) == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

# Preprocessing Data

In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [21]:
label_map = {label: num for num, label in enumerate(actions)}

In [24]:
label_map

{'Hello': 0, 'ThankYou': 1, 'ILoveYou': 2}

In [26]:
sequences, labels = [],[]

for action in actions:
    for seq in range(no_sequence):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(seq), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

FileNotFoundError: [Errno 2] No such file or directory: 'MP_DATA\\Hello\\0\\0.npy'

In [30]:
np.array(sequences).shape

(0,)

In [32]:
np.array(labels).shape

(0,)

In [33]:
X = np.array(sequences)

In [35]:
y = to_categorical(labels).astype(int)

In [37]:
X_train, X_test, y_train, t_test = train_test_split(X,y,test_size=0.05)

# Building Neural Network 

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [41]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir = log_dir)

In [43]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences = True, activation = 'relu'))
model.add(LSTM(64, return_sequences = True, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [45]:
res = [.7,0.2,0.1]
actions[np.argmax(res)] #np.argmax return the index of the max element in the array

'Hello'

In [46]:
model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [48]:
model.fit(X_train, y_train, epochs = 2000, callbacks = [tb_callback])

In [50]:
model.summary()

# Lets Predict

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[0])]

In [ ]:
actions[np.argmax(y_test[0])]

# Saving the Model

In [ ]:
model.save('Sign_Language_Predictor.model')

In [ ]:
del model #this will delete the trained model

In [ ]:
model.load_weights('Sign_Language_Predictor.model')

# Evaluating using Confusion Matrix

In [51]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [53]:
ytrue = np.argmax(y_test, axis = 1).tolist()
yhat = np.argmax(yhat, axis = 1).tolist()

In [ ]:
multilabel_confusion_matrix(yture, yhat)

In [ ]:
accuracy_score(ytrue,yhat)

# Ending... Testing in real 

In [59]:
colors = [(245,117,16),(117,245,16),(16,117,245)]
def prob_stylish_visualizer(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0,85+num*40),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
sequence = []
sentence = []
thresold = 0.4



cap = cv2.VideoCapture(0)

#Access mediapipe model
with mp_holistics_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        success, frame = cap.read()

        img, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(img,results)
        
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis = 0))[0]
            print(res)
        
        
        if res[np.argmax(res)] > thresold:  #max value is > thresold which implies max possibility
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append[actions[np.argmax(res)]]
            else:
                sentence.append(actions[np.argmax(res)])
                
            #if we get the same word we skip it and do not append, only append if word is diff or empty list
            
        
        if len(sentence) > 5:
            sentence = sentence[-5:]
            
        image = prob_stylish_visualizer(res, actions, image, colors)
        
        cv2.rectangle(img, (0,0), (640,40), (245,117,16), -1)
        cv2.putText(img, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        
        frame = cv2.resize(frame, (860,640))
        cv2.imshow("Output", img)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [55]:
# """
# our model is expecting dimension of - (nums_sequence, 30, 1662)
# if we give directly X_test(which is of shape (30,1662) it will return error)
# therefore I have used np.expand_dims(X_test[0], axis = 0) , as it converts the shape to (1 ,30 , 1662)
# """

In [57]:
#test both

# np.expand_dims(X_test[0], axis = 1)
np.expand_dims(X_test[0], axis = 0)
